# Machine Learning: Regression

Unter der Regression versteht man die Vorhersage von *kontinuierlichen Werten*. Ganz einfache Beispiele dafür kennst du aus deinem täglichen Leben - so nutzt z.B. die Restlaufzeitanzeige von Handy lineare Regression genau die die Reichweitenanzeige im Auto.

Wir konzentrieren uns hier auf die Regressionsanalyse im Heise Newsticker.

## Regressionsanalyse des Heise Newstickers

### Lineare Regression mit `scipy`

Im letzten Teil hast du schon Statistik-Informationen zum Heise-Newsticker betrachtet. Nun arbeitest du wieder mit den gleichen Daten und führst eine Regressionanalyse durch.

Du weißt schon, wie du die Daten mit `pandas` einlesen kannst:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-monthly.csv || wget  https://github.com/heiseacademy/ml-python/blob/main/02-ml-intro/heise-monthly.csv")

In [ ]:
import pandas as pd
heise_monthly = pd.read_csv("heise-monthly.csv", parse_dates=["month"], index_col="month")

Vergegenwärtige dir nochmal die Zeitentwicklung:

In [ ]:
heise_monthly["count"].plot()

Nun verwendest du `scipy`, um die Daten mittels [linearer Regression](https://de.wikipedia.org/wiki/Lineare_Regression) zu fitten:

In [ ]:
from scipy.stats import linregress
lrc = linregress(range(len(heise_monthly)), heise_monthly["count"].values)
lrc

Das Pearson-R sagt die, wie "stark" der Zusammenhang ist und der *p-Wert*, wie verlässlich der Fit ist. Dazu sollte dieser möglichst klein sein, was hier schon ganz gut klappt.

Integriere nun die Regressionsgerade noch in den `DataFrame`:

In [ ]:
heise_monthly["predict_count"] = [i*lrc.slope+lrc.intercept for i in range(len(heise_monthly))]

Und lass dir das Ergebnis grafisch ausgeben:

In [ ]:
heise_monthly[["count", "predict_count"]].plot()

Das sieht ganz gut aus.

Betrachte zum Vergleich noch die Länge der Artikel:

In [ ]:
lrt = linregress(range(len(heise_monthly)), heise_monthly["tokens"].values)
lrt

Du siehst, dass sowohl das Pearson-R näher an 1 liegt als auch der *p-Wert* viel kleiner ist. Der Fit ist also besser.

In [ ]:
heise_monthly["predict_tokens"] = [i*lrt.slope+lrt.intercept for i in range(len(heise_monthly))]

In [ ]:
heise_monthly[["tokens", "predict_tokens"]].plot()

Wie du schon vermutet hast, ist die Approximation besser. Dennoch vermisst du einiges, z.B. ein Fehlerkorridor. Außerdem sind mögliche saisonale Effekte nicht berücksichtigt. Dazu reicht dir eine lineare Regression nicht aus.

`scikit-learn` bietet dir hier viel mehr Möglichkeiten, die du dir nun anschaust:

### Lineare Regression mit `scikit-learn`

In [ ]:
from sklearn import linear_model
slrt = linear_model.LinearRegression()
X = [[i] for i in range(len(heise_monthly))]
Y = heise_monthly["tokens"].values
slrt.fit(X, Y)

In [ ]:
heise_monthly["predict_tokens_sklearn_linear"] = slrt.predict(X)

In [ ]:
heise_monthly[["tokens", "predict_tokens_sklearn_linear"]].plot()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
def print_scores(ground_truth, predict):
    print('Mittlerer quadratischer Fehler', mean_squared_error(ground_truth, predict))
    # 1 entspricht einer 100% Vorhersage
    print('Coefficient of determination: %.2f' % r2_score(ground_truth, predict))
    
print_scores(Y, heise_monthly["predict_tokens_sklearn_linear"])

Allerdings ist der Fehler jetzt in den *Trainingdaten* berechnet. Wenn du Machine Learning betreibst, interessiert du dich für *Vorhersagen* von unbekannten Werten. Deswegen wendest du jetzt einen Trick an und teilst die Datenmenge (mit den bekannten Werten) in zwei Teilmengen auf, eine *Trainingsdatenmenge* und eine *Testdatenmenge*. 

Der `fit`-Funktion gibst du nur die Trainingsdaten, die Vorhersage lässt du für die Testdaten durchführen. Da du dich für Vorhersagen *zukünftiger Daten* interessierst, nutzt du als Trainingsdaten die alten Werte, die Testdaten sind die neueren:

In [ ]:
(X_train, X_test) = (X[:-50], X[-50:])
(Y_train, Y_test) = (Y[:-50], Y[-50:])
slrt.fit(X_train, Y_train)

In [ ]:
print_scores(Y_test, slrt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_linear"] = slrt.predict(X)

In [ ]:
heise_monthly["predict_tokens_sklearn_linear_train"] = list(slrt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_linear_test"] = [None]*len(X_train) + list(slrt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_linear_train", "predict_tokens_sklearn_linear_test"]].plot()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dtt = DecisionTreeRegressor(max_depth=4)

dtt.fit(X_train, Y_train)
print_scores(Y_test, dtt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_dt_train"] = list(dtt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_dt_test"] = [None]*len(X_train) + list(dtt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_dt_train", "predict_tokens_sklearn_dt_test"]].plot()

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

abt = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=42)

abt.fit(X_train, Y_train)
print_scores(Y_test, abt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_ab_train"] = list(abt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_ab_test"] = [None]*len(X_train) + list(abt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_ab_train", "predict_tokens_sklearn_ab_test"]].plot()

In [ ]:
from sklearn import ensemble
gbt = ensemble.GradientBoostingRegressor()
gbt.fit(X_train, Y_train)
print_scores(Y_test, gbt.predict(X_test))

In [ ]:
heise_monthly["predict_tokens_sklearn_gb_train"] = list(gbt.predict(X_train)) + [None]*len(X_test)
heise_monthly["predict_tokens_sklearn_gb_test"] = [None]*len(X_train) + list(gbt.predict(X_test))
heise_monthly[["tokens", "predict_tokens_sklearn_gb_train", "predict_tokens_sklearn_gb_test"]].plot()

## Bessere Vorhersage von zukünftigen Werten

Dafür gibt es spezielle Pakete wie [`prophet`](https://facebook.github.io/prophet/).

In [ ]:
!pip install prophet

In [ ]:
from prophet import Prophet

Die Daten müssen dazu ein bisschen anders vorbereitet werden, du brauchst einen speziellen `DataFrame`:

In [ ]:
pa = pd.DataFrame()
pa["ds"] = heise_monthly.index.values
pa["y"] = heise_monthly["tokens"].values
pa

Dann instanziierst du die `Prophet`-Klasse:

In [ ]:
m = Prophet()
m.fit(pa)

Und erzeugst einen `DataFrame` für die zukünftigen Werte. Hier benötigst du eine *monatliche Frequenz*:

In [ ]:
future = m.make_future_dataframe(periods=20, freq='M')

Jetzt kannst du die Vorhersage durchführen. `yhat` ist der wahrscheinlichste Wert, `Prophet` liefert dir auch noch einen *Korridor*:

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Die Daten kannst du sehr einfach visualisieren und seihst, dass das Ergebnis besser ist als unsere einfach Regression oben:

In [ ]:
m.plot(forecast)

`Prophet` kann nocht mehr, wie etwa Langfristtrends oder saisonale Schwankungen berechnen:

In [ ]:
m.plot_components(forecast)

## Anspruchsvolle Regression

Wie du oben siehst, hat die lineare Regression gut funktioniert. Die weit fortschrittlicheren Verfahren haben hingegen nur konstante Werte für die Vorhersage geliefert und waren damit nicht sinnvoll einsetzbar.

Eigene Modelle musst du auf jeden Fall genau testen, bevor du sie einsetzt. Oft ist es daher günstig, wenn du ein schon existierendes Modell verwendest.